### Day 22

### Part 1:
- Reactor core
    - 3D grid of cubes
    - Each is on or off, and start off
    - Input is a set of volumes to turn on or off
        - e.g. "on x=10..12,y=10..12,z=10..12"
    - Only consider x,y,z between -50 and 50
    - Apply all instructions and then count number of cubes that are on

Thoughts:
- Make a 100x100x100 list? 0 for off, 1 for on? Probably won't work for whatever part 2 is
- A single dictionary with the positions that are on?
- This would be trivial with a 100x100x100 numpy array

In [1]:
from collections import defaultdict
class ReactorCore(object):
    def __init__(self):
        """Represents a reactor core.
        Has self.core, which is a default dictionary so we don't have to track every square in memory
        """
        self.core = defaultdict(int)
        self.onoff_dict = {"on":1,"off":0}
        
        self.xrange = [-50,50]
        self.yrange = [-50,50]
        self.zrange = [-50,50]
        
    def switch_cuboid_area(self, xr:list, yr:list, zr:list, value:int):
        """Turn a volume bounded by xr = [x1,x2], yr=[y1,y2], zr=[z1,z2] on or off (determined by value).
        Assumes the coordinates are all within the bounds of the reactor."""
        for x in range(xr[0],xr[1]+1):
            for y in range(yr[0],yr[1]+1):
                for z in range(zr[0],zr[1]+1):
                    self.core[(x,y,z)] = value
                    
    def check_edges(self,xr,yr,zr):
        """Check if these coordinates are in the grid"""
        # Is it completely outside of the grid?
        # Check X
        if xr[1] < self.xrange[0] or xr[0] > self.xrange[1]:
            return False,0,0,0
        # Check Y
        elif yr[1] < self.yrange[0] or yr[0] > self.yrange[1]:
            return False,0,0,0
        # Check Z
        elif zr[1] < self.zrange[0] or zr[0] > self.zrange[1]:
            return False,0,0,0
        else:
            # Then at least some part is within the grid
            # Align the min and max values with the edge of the grid
            xr = [max(self.xrange[0],xr[0]),min(self.xrange[1],xr[1])]
            yr = [max(self.yrange[0],yr[0]),min(self.yrange[1],yr[1])]
            zr = [max(self.zrange[0],zr[0]),min(self.zrange[1],zr[1])]
            
            return True, xr, yr, zr
        
    def parse_command(self,command):
        """ 
        Input:
            command like "on x=10..12,y=10..12,z=10..12"
        Output:
            xr,yr,zr,on_or_off
            """
        # Convert from text to coordinates and an on or off command
        onoff,coords = command.split(" ")
        onoff = self.onoff_dict[onoff]
        xr,yr,zr = coords.split(",")

        xr = xr.replace("x=","").split("..")
        xr = [int(x) for x in xr]

        yr = yr.replace("y=","").split("..")
        yr = [int(y) for y in yr]

        zr = zr.replace("z=","").split("..")
        zr = [int(z) for z in zr]
        
        return xr,yr,zr,onoff
                    
    def process_command(self,command):
        """command is e.g. "on x=10..12,y=10..12,z=10..12" """
        
        xr,yr,zr,onoff = self.parse_command(command)
        
        # Check edges
        is_in_grid, xr, yr, zr = self.check_edges(xr,yr,zr)
        
        if is_in_grid:
            self.switch_cuboid_area(xr,yr,zr,onoff)
        else:
            pass
            
    def process_file(self,fname):
        with open(fname, "r") as f:
            data = f.read().splitlines()
            
        for command in data:
            self.process_command(command)

    def count_on_cubes(self):
        n_on = 0
        for coords, onoff in self.core.items():
            n_on += onoff
        return n_on
        
        

In [2]:
core = ReactorCore()
# command = "on x=10..12,y=10..12,z=10..12"
# core.process_command(command)
core.process_file("inputs/day22_test_input.dat")
core.count_on_cubes()

590784

In [3]:
core = ReactorCore()
# command = "on x=10..12,y=10..12,z=10..12"
# core.process_command(command)
core.process_file("inputs/day22_input.dat")
core.count_on_cubes()

647062

### Part 2
- Now it's an infinite grid

Thoughts:
- Volumes are too large for the approach above, since we can't record all of the points
- Maybe record all of the commands, then go through the "on" commands and look for any other commands that cancel them out. Otherwise count them and move on.
- Loop through from start to end:
    - Make a cube for the new command
    - If this cube completely encloses any stored cubes, then delete the stored cube
    - Otherwise, if this cube intersects any stored cubes, then split the stored cube along one of the intersection axes
    - Repeat until there are no more enclosures or intersections. Then add the new cube if it's "on" or delete it if "off"
    - Then calculate the total volume of all cubes
    
How to split:
- cube is x1-x2, y1-y2, z1-z2
- want to remove x3-x4, y3-y4, z3-z4
- Consider the first cube as:
    - (x1-x3),(y1-y3),(z1-z3) 000
    - (x1-x3),(y1-y3),(z4-z2) 001
    - (x1-x2),(y4-z2),(z1-z3) 010
    - (x1-x2),(y4-z2),(z4-z2) 011
    - (x4-x2),(y1-y3),(z1-z3) 100
    - (x4-x2),(y1-y3),(z4-z2) 101
    - (x4-x2),(y4-y2),(z1-z3) 110
    - (x4-x2),(y4-y2),(z4-z2) 111



In [4]:
class Cuboid(object):
    def __init__(self,xr,yr,zr,onoff):
        self.xr = [min(xr),max(xr)]
        self.yr = [min(yr),max(yr)]
        self.zr = [min(zr),max(zr)]
        
        self.onoff = onoff
        
    def __eq__(self,c2):
        """Custom equals operator so I can write some tests."""
        return self.unpack() == c2.unpack()
    
    def __len__(self):
        return 1
        
    def unpack(self):
        return self.xr,self.yr,self.zr,self.onoff
        
    def volume(self):
        """Return volume of this cuboid"""
        return (self.xr[1]-self.xr[0]+1)*(self.yr[1]-self.yr[0]+1)*(self.zr[1]-self.zr[0]+1)  
    
    def check_enclosure_1d(self,xr1,xr2):
        """Check if a range xr2=[x3,x4] encloses a range xr1=[x1,x2]"""
        return (xr2[0] <= xr1[0] <= xr2[1]) and (xr2[0] <= xr1[1] <= xr2[1])
        
    def check_intersection_1d(self,xr1,xr2):
        """Check if a range xr2=[x3,x4] intersects a range xr1=[x1,x2]"""
        if (xr1[0] <= xr2[0] <= xr1[1]) or (xr1[0] <= xr2[0] <= xr1[1]):
            return True
        elif (xr1[0] <= xr2[1] <= xr1[1]) or (xr1[0] <= xr2[1] <= xr1[1]):
            return True
        else:
            return False    
    
    def check_and_split(self,cuboid2):
        """Compare this cuboid with cuboid2.
        1. If they are the same, return cuboid2
        2. If cuboid2 encloses this cuboid, return cuboid2.
        3. If cuboid2 intersects this cuboid, split it and return the new cuboids along with cuboid2.
        4. Otherwise return this cuboid and cuboid2.
        """
        
        # Unpack
        xr1,yr1,zr1,onoff1 = self.unpack()
        xr2,yr2,zr2,onoff2 = cuboid2.unpack()
        
        # Check enclosure in each axis
        x_enc = self.check_enclosure_1d(xr1,xr2)
        y_enc = self.check_enclosure_1d(yr1,yr2)
        z_enc = self.check_enclosure_1d(zr1,zr2)
        
        # If total enclosure, return only cuboid 2
        if x_enc and y_enc and z_enc:
            return cuboid2
        
        # Otherwise check if they intersect
        x_overlap = self.check_intersection_1d(xr1,xr2)
        y_overlap = self.check_intersection_1d(yr1,yr2)
        z_overlap = self.check_intersection_1d(zr1,zr2)
        
        # If there's an overlap or intersection in each axis then we need to split them
        if (x_overlap or x_enc) and (y_overlap or y_enc) and (z_overlap or z_enc):
            if x_overlap and (not x_enc):
                # Can we split on the left?
                if xr1[0] < xr2[0]:
                    c1 = Cuboid([xr1[0],xr2[0]-1],yr1,zr1,onoff1)
                    c2 = Cuboid([xr2[0],xr1[1]],yr1,zr1,onoff1)
                elif xr1[1] > xr2[1]:
                    # Then split on the right
                    c1 = Cuboid([xr1[0],xr2[1]],yr1,zr1,onoff1)
                    c2 = Cuboid([xr2[1]+1,xr1[1]],yr1,zr1,onoff1)
                else:
                    print("problemx:",xr1,xr2)
            elif y_overlap and (not y_enc):
                if yr1[0] < yr2[0]:
                    c1 = Cuboid(xr1,[yr1[0],yr2[0]-1],zr1,onoff1)
                    c2 = Cuboid(xr1,[yr2[0],yr1[1]],zr1,onoff1)
                elif yr1[1] > yr2[1]:
                    # Then split on the right
                    c1 = Cuboid(xr1,[yr1[0],yr2[1]],zr1,onoff1)
                    c2 = Cuboid(xr1,[yr2[1]+1,yr1[1]],zr1,onoff1)
                else:
                     print("problemy:",yr1,yr2)
            elif z_overlap and (not z_enc):
                if zr1[0] < zr2[0]:
                    c1 = Cuboid(xr1,yr1,[zr1[0],zr2[0]-1],onoff1)
                    c2 = Cuboid(xr1,yr1,[zr2[0],zr1[1]],onoff1)
                elif zr1[1] > zr2[1]:
                    # Then split on the right
                    c1 = Cuboid(xr1,yr1,[zr1[0],zr2[1]],onoff1)
                    c2 = Cuboid(xr1,yr1,[zr2[1]+1,zr1[1]],onoff1)
                else:
                     print("problemz:",zr1,zr2)
            else:
                print("Not an overlap or an enclosure?")
                    
            return c1,c2,cuboid2
        
        # Otherwise there's no overlap so we can keep this cuboid intact
        return self,cuboid2

In [5]:
### Test 1: Same cube, just return the new one only
c = Cuboid([0,1],[0,1],[0,1],0)
d = Cuboid([0,1],[0,1],[0,1],1)
assert d == c.check_and_split(d)

### Test 2: Enclosure. Should return the new one only
c = Cuboid([0,1],[0,1],[0,1],0)
d = Cuboid([-1,2],[0,2],[0,2],1)
assert d == c.check_and_split(d)
### Test 2b: Enclosure but with shared side. Should return the new one only
c = Cuboid([0,2],[0,2],[0,2],0)
d = Cuboid([-1,2],[0,2],[0,2],1)
assert d == c.check_and_split(d)

### Test 3: Intersection. Should return 3 cuboids, second and third are the same
c = Cuboid([0,2],[0,2],[0,2],1)
d = Cuboid([1,2],[0,2],[0,2],1)
x = c.check_and_split(d)
assert len(x) == 3
assert x[0] == Cuboid([0,0],[0,2],[0,2],1)
assert x[1] == x[2] == Cuboid([1,2],[0,2],[0,2],1)

### Test 4: Intersection. Should return 3 cuboids, first and third are the same
c = Cuboid([0,2],[0,2],[0,2],1)
d = Cuboid([0,1],[0,2],[0,2],1)
x = c.check_and_split(d)
assert len(x) == 3
assert x[0] == x[2] == Cuboid([0,1],[0,2],[0,2],1)
assert x[1] == Cuboid([2,2],[0,2],[0,2],1)

In [6]:
class InfiniteReactorCore(ReactorCore):
    def __init__(self):
        self.cubes_on = 0
        self.onoff_dict = {"on":1,"off":0}
        self.cubes = []
        
    def load_instructions(self,fname):
        with open(fname, "r") as f:
            data = f.read().splitlines()
            
        self.commands = []
        for cmd in data:
            xr,yr,zr,onoff = self.parse_command(cmd)
            self.commands.append([xr,yr,zr,onoff])
            
    def run_one_command(self):
        """Execute one command from the command list"""
        cmd = self.commands.pop(0)
        
        xr,yr,zr,onoff = cmd
        cuboid2 = Cuboid(xr,yr,zr,onoff)
        
        # Repeatedly check through the existing cubes to make sure there's no issues
        next_cube_list = self.cubes
        found_issue = True
        
        while found_issue:
            new_cube_list = next_cube_list
            next_cube_list = []
            
            found_issue = False # reset
            for cuboid in new_cube_list:
                new_cubes = cuboid.check_and_split(cuboid2)

                if len(new_cubes) == 1:
                    # The new cube enclosed the old one so ignore the old one
                    found_issue = True
                elif len(new_cubes) == 2:
                    # No issue!
                    c1,_ = new_cubes
                    next_cube_list.append(c1)
                    pass
                else:
                    # Split the original cube into 2
                    c1,c2,_ = new_cubes
                    next_cube_list.extend([c1,c2])
                    found_issue = True
                    
        # We should have no overlaps or issues now
        # Add the new cube to the list if it's "on"
        if onoff == 1:
            next_cube_list.append(cuboid2)
        
        self.cubes = next_cube_list
    
    def run_all_commands(self):
        while len(self.commands) > 0:
            self.run_one_command()
    
    def count_on_cubes(self):
        v = 0
        for cube in self.cubes:
            v += cube.volume()
        return v
    
    def print_cubes(self):
        for ix,c in enumerate(self.cubes):
            print(ix,c.unpack())


In [7]:
core = InfiniteReactorCore()
core.load_instructions("inputs/day22_test_input_pt2.dat")
core.run_all_commands()
core.count_on_cubes()

2758514936282235

In [ ]:
core = InfiniteReactorCore()
core.load_instructions("inputs/day22_input.dat")
core.run_all_commands()
core.count_on_cubes()